# Clustering StackOverflow Q&A

EPFL Big Data Analysis Week 2 Assignment
https://www.coursera.org/learn/scala-spark-big-data/home/info

"The overall goal of this assignment is to implement a distributed k-means algorithm which clusters posts on the popular question-answer platform StackOverflow according to their score. Moreover, this clustering should be executed in parallel for different programming languages, and the results should be compared.

The motivation is as follows: StackOverflow is an important source of documentation. However, different user-provided answers may have very different ratings (based on user votes) based on their perceived value. Therefore, we would like to look at the distribution of questions and their answers. For example, how many highly-rated answers do StackOverflow users post, and how high are their scores? Are there big differences between higher-rated answers and lower-rated ones?"

Data file download link: http://alaska.epfl.ch/~dockermoocs/bigdata/stackoverflow.csv

**WORK IN PROGRESS**

In [1]:
import time

# Credits to Fahim Sakri 
# Source (https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d)
# An annotation for timing a python function
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ("%r  %2.2f ms" % (method.__name__, (te - ts) * 1000))
        return result
    return timed

from post import Post

In [11]:
## Setup
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext
import pandas as pd
import numpy as np

spark = SparkSession \
    .builder \
    .appName("EPFL Wk2 Assignment") \
    .config("spark.executor.instances", 4) \
    .config("spark.executor.cores", 4) \
    .config("spark.cores.max", 4) \
    .config("spark.executor.memory", "4G")\
    .config("spark.driver.memory", "12G")\
    .config("spark.driver.maxResultSize", "10G")\
    .getOrCreate()
        
# Create RDD
data = spark.read.csv('/data/epfl-big-data-analysis/stackoverflow.csv', header=False, inferSchema=True)
data = data.withColumnRenamed("_c0", "post_type_id") # type 1 = question, type 2 = answer
data = data.withColumnRenamed("_c1", "id")
data = data.withColumnRenamed("_c2", "acceptedAnswerId")
data = data.withColumnRenamed("_c3", "parentId")
data = data.withColumnRenamed("_c4", "score")
data = data.withColumnRenamed("_c5", "tag")

#data = pd.read_csv('/data/epfl-big-data-analysis/stackoverflow.csv', 
#                   names=["post_type_id", "id", "acceptedAnswerId", "parentId", "score", "tag"],
#                   dtype={'post_type_id': np.int64, 'score': np.float16})
data_size = data.count()

In [12]:
data.select('tag').distinct().collect()
print(data_size)

8143801


In [16]:
sample_size = int(data_size/100)
print(sample_size)
posts = spark.sparkContext.parallelize(data.head(sample_size)).filter(lambda p: p.tag is not None or p.post_type_id == 2)
#langs = ["JavaScript", "Java", "PHP", "Python", "C#", "C++", "Ruby", "CSS",
#"Objective-C", "Perl", "Scala", "Haskell", "MATLAB", "Clojure", "Groovy"]
print(posts.take(1))
#posts.filter(lambda p: p.post_type_id == 1)
langs = posts.map(lambda p: p.tag).distinct().collect()
print(langs)

81438
[Row(post_type_id=1, id=27233496, acceptedAnswerId=None, parentId=None, score=0, tag='C#')]
[None, 'Java', 'Python', 'JavaScript', 'CSS', 'Groovy', 'C++', 'Objective-C', 'Haskell', 'Ruby', 'Scala', 'MATLAB', 'Clojure', 'PHP', 'Perl', 'C#']


### Step 1 Preparation - Grouped posts by question
First we use the map function to create kv pairs for each type of posts namely questions and answers.  
Then a join operation is used for merging the two datasets.  A dataset `RDD[(QID, Iterable(Question, Answer))]` should be useful, the key is the ID of the question post and the values is a collection of tuple (Question, Answer).

In [17]:
questions = posts.filter(lambda p: p.post_type_id == 1).map(lambda p: (p.id, p))
answers = posts.filter(lambda p: p.post_type_id == 2).map(lambda p: (p.parentId, p))
grouped = questions.join(answers).groupByKey() # Use inner join to exclude posts with no answers
print(questions.take(1));
print(answers.take(1));
print(grouped.take(2));

[(27233496, Row(post_type_id=1, id=27233496, acceptedAnswerId=None, parentId=None, score=0, tag='C#'))]
[(5484340, Row(post_type_id=2, id=5494879, acceptedAnswerId=None, parentId=5484340, score=1, tag=None))]
[(21984912, <pyspark.resultiterable.ResultIterable object at 0x7faf641af9b0>), (12108144, <pyspark.resultiterable.ResultIterable object at 0x7faf641af9e8>)]


### Step 2 Calculate maximum answer score for each question
Produce a set of key-value pairs - Key of the pair is the question and value should be the maximum answer score of the question.  The output is an `RDD[(Posting, Int)]`

In [39]:
def post_max_scores(iterable):
    max_score = -1
    for pair in iterable:
        question = pair[0]
        answer_score = pair[1].score
        if answer_score > max_score:
            max_score = answer_score
        return (question, max_score)

def scoredPostings(question_scores):
    return question_scores.values().map(post_max_scores)

#scored = scoredPostings(grouped)
#print(scored.take(2))

### Step 3 Create vectors for clustering
Prepare the vectors as an input for clustering.  

<br/>
Index of the language (in the langs list) multiplied by the `langSpread` factor.

The highest answer score (computed above).

The `langSpread factor` is provided (set to 50000). Basically, it makes sure posts about different programming languages have at least distance 50000 using the distance measure provided by the euclideanDist function. You will learn later what this distance means and why it is set to this value. The output is `RDD[(Int, Int)]`

In [20]:
langSpread = 50000

def vectorPostings(scores):
    # Prepare the vectors as an input for clustering
    return scores.map(lambda s: (langs.index(s[0].tag)*langSpread, s[1]))

#vectors = vectorPostings(scored)
#print(vectors.count())
#print(vectors.take(1))

In [22]:
## Helper methods for clustering
from scipy.spatial import distance

def euclideanDistanceSum(v1, v2):
    distance_sum = 0
    for i in np.arange(0, len(v1)):
        vector1 = v1[i]
        vector2 = v2[i]
        distance_sum = distance_sum + distance.euclidean(vector1, vector2)
    return distance_sum

def findClosest(p, means):
    closest_distance = np.iinfo(np.int32).max
    for i in np.arange(0, len(means)):
        m = means[i]
        dist = distance.euclidean(p, m)
        print('i:' + str(dist))
        if dist < closest_distance:
            closest_i = i
            closest_distance = dist
    return closest_i

def averageVectors(v1, v2):
    return ((v1[0] + v2[0])/2, (v1[1] + v2[1])/2)

def distance_from_mean(v, means):
    mean_index = v[0]
    mean_vector = means[mean_index]
    return distance.euclidean(v[1], mean_vector)
    
def averageVectors1(v):
    sum1 = 0
    sum2 = 0
    for vector in v:
        sum1 = sum1 + vector1
        sum2 = sum2 + vector[1]
    return (sum1/len(v), sum2/len(v))


In [69]:
def kmeans(kmeansEta, kmeansMaxIterations, kmeansKernels, vectors, debug=False):
    distinct_vectors = vectors.distinct()
    sampleMeansRatio = kmeansKernels/distinct_vectors.count()
    means = distinct_vectors.sample(False, sampleMeansRatio).collect()
    print("Number of means:" + str(len(means)))
    return kmeans_itr(kmeansEta, kmeansMaxIterations, means, vectors, debug)


def kmeans_itr(kmeansEta, kmeansMaxIterations, means, vectors, debug=False):    
    itr = 1
    print('Vectors size:' + str(vectors.count()))
    print('Means size:' + str(len(means)))
    while itr <= kmeansMaxIterations:
        clusters = vectors.map(lambda v: (findClosest(v, means), v)).cache()
        #if debug:
            #print("Clusters assignment:" + str(clusters.keys().collect()))
        error = clusters.map(lambda v: (v[0], distance_from_mean(v, means))) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .collect()

        averaged = clusters.reduceByKey(lambda v1, v2: averageVectors(v1, v2)).sortByKey()
        mm = averaged.values().collect()
        if debug:
            print("New means keys:" + str(averaged.keys()))

        if (len(means) != len(mm)):
            raise Exception("Number of new means (" + str(len(mm)) +") is not the same as the number of old means(" + str(len(means)) + ")")

        d_sum = euclideanDistanceSum(means, mm)
        means = mm
        
        # Calculate distance from mean for each vector
        clusters_mean = clusters.join(mm).collect()
        
        
        print("Iteration " + str(itr) + ' completed, convergence=' + str(d_sum))

        if d_sum <= kmeansEta:
            break

        itr = itr + 1
        
    if debug:
        print('means:')
        print(means)
        print('error:')
        print(error)
        print('euclideanDistanceSum:' + str(d_sum))
    
    if itr > kmeansMaxIterations:
        print('Failed to converge after ' + str(kmeansMaxIterations) + ' iterations.')
    else:
        print('Converged after ' + str(itr-1) + ' iterations. Error=' + str(d_sum))
    return {'means':means, 'clusters':clusters.collect()}


In [24]:
# Test kmeans
def testKmeans():
    
    # An example online
    v1 = [18, 21, 22, 24, 26, 26, 27, 30, 31, 35, 39, 40, 41, 42, 44, 46, 47, 48, 49, 54]
    v2 = [10, 11, 22, 15, 12, 13, 14, 33, 39, 37, 44, 27, 29, 20, 28, 21, 30, 31, 23, 24]

    vv = list()
    for ii in np.arange(0, len(v1)):
        vv.append((v1[ii], v2[ii]))

    my_vectors = spark.sparkContext.parallelize(vv)    
    my_means = my_vectors.takeSample(False, 3)

    #kmeans(1, 10, my_means, my_vectors)
    
    kmeans(20, 120, my_means, my_vectors, True)



In [77]:
## K-means parameter: Number of clusters
kmeansKernels = 45
## K-means parameter: Convergence criteria
kmeansEta = 20
## K-means parameter: Maximum iterations
kmeansMaxIterations = 130

def cluster(data):
    my_vectors = vectorPostings(scoredPostings(data))
    kmeans(kmeansEta, kmeansMaxIterations, 45, my_vectors)
    
# Cluster a small sample 
sample = spark.sparkContext.parallelize(grouped.takeSample(False, 20000))
vectors = vectorPostings(scoredPostings(sample))
distinct_vectors = vectors.distinct()
sampleMeansRatio = kmeansKernels/distinct_vectors.count()
means = distinct_vectors.sample(False, sampleMeansRatio).collect()
#kmeans_itr(kmeansEta, kmeansMaxIterations, means, vectors, True)
clusters = vectors.map(lambda v: (findClosest(v, means), v)).cache()
averaged = clusters.reduceByKey(lambda v1, v2: averageVectors(v1, v2)).sortByKey()
#results = kmeans(sample)

#spark.sparkContext.parallelize(grouped.takeSample(False, 20000))
#distinct_vectors = vectors.distinct()
#sampleMeansRatio = kmeansKernels/distinct_vectors.count()
#means = distinct_vectors.sample(False, sampleMeansRatio).collect()

#results = cluster(grouped)

In [110]:
print(clusters.keys().take(1))
print(averaged.keys().take(1))
merged = clusters.join(averaged)
#merged.take(2)
merged.mapValues(lambda v : v[0]).take(1).apply(lambda k: )

[14]
[0]


[(0, (100000, 2))]

In [80]:
results['errors']

[(0, 17750214.891628295),
 (1, 7942297.838934626),
 (2, 935.8513609692402),
 (3, 10132891.141639356),
 (4, 844.0286521818489),
 (5, 895.1356080605121),
 (6, 42.196533203125)]

Based on these initial means, and the provided variables converged method, implement the K-means algorithm by iteratively:

pairing each vector with the index of the closest mean (its cluster);

computing the new means by averaging the values of each cluster.

To implement these iterative steps, use the provided functions findClosest, averageVectors, and euclideanDistance.

Note 1:

In our tests, convergence is reached after 44 iterations (for langSpread=50000) and in 104 iterations (for langSpread=1), and for the first iterations the distance kept growing. Although it may look like something is wrong, this is the expected behavior. Having many remote points forces the kernels to shift quite a bit and with each shift the effects ripple to other kernels, which also move around, and so on. Be patient, in 44 iterations the distance will drop from over 100000 to 13, satisfying the convergence condition.

If you want to get the results faster, feel free to downsample the data (each iteration is faster, but it still takes around 40 steps to converge):

val scored  = scoredPostings(grouped).sample(true, 0.1, 0)
However, keep in mind that we will test your assignment on the full data set. So that means you can downsample for experimentation, but make sure your algorithm works on the full data set when you submit for grading.

Note 2:

The variable langSpread corresponds to how far away are languages from the clustering algorithm's point of view. For a value of 50000, the languages are too far away to be clustered together at all, resulting in a clustering that only takes scores into account for each language (similarly to partitioning the data across languages and then clustering based on the score). A more interesting (but less scientific) clustering occurs when langSpread is set to 1 (we can't set it to 0, as it loses language information completely), where we cluster according to the score. See which language dominates the top questions now?